In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

In [58]:
def NSW(xs):
    return -np.prod(xs*np.array([1,2,40]))

In [53]:
xs = np.array([1/3, 1/3, 1/3])
xs = np.array([1/3, 1/3, 1/3])
NSW(xs)

-0.2962962962962963

In [56]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

In [59]:
res = minimize(NSW, start_pos, method='SLSQP', bounds=bnds ,constraints=cons)
res

     fun: -2.962962870208538
     jac: array([-8.88880694, -8.88536593, -8.89249241])
 message: 'Optimization terminated successfully'
    nfev: 29
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.33327282, 0.33339064, 0.33333654])

In [76]:
alpha = 3
def F(x):
    costs = [1., 2., 3.]
    vals  = np.array([costs[i]*np.power(x[i], 1+1/alpha) for i in range(len(costs))])
    return np.sum(vals/(1+1/alpha))

In [77]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 0.7133805647207109
     jac: array([0.9511741, 0.9511741, 0.9511741])
 message: 'Optimization terminated successfully'
    nfev: 41
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.86055778, 0.10756972, 0.0318725 ])

In [78]:
def theos(costs):
    den = np.sum([np.power(costs[i], -alpha) for i in range(len(costs))])
    return [np.power(costs[i], -alpha) / den for i in range(len(costs))]

In [79]:
theos([1.,2.,3.])

[0.8605577689243028, 0.10756972111553785, 0.03187250996015936]